In [36]:
# libraries and credential

import pandas as pd
import uuid
from google.cloud import bigquery


service_account_path = r'..\bot-recommendation-0568d5fe0b88.json'

file_1 = r"..\data\raw\amz_uk_processed_data.csv"
df_amz_uk = pd.read_csv(file_1)

In [24]:
print(df_amz_uk.shape)
print(df_amz_uk.info())
print(df_amz_uk['boughtInLastMonth'].value_counts())


(2222742, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2222742 entries, 0 to 2222741
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   asin               object 
 1   title              object 
 2   imgUrl             object 
 3   productURL         object 
 4   stars              float64
 5   reviews            int64  
 6   price              float64
 7   isBestSeller       bool   
 8   boughtInLastMonth  int64  
 9   categoryName       object 
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 154.7+ MB
None
boughtInLastMonth
0        2061427
50         63601
100        44173
200        17509
300         9523
400         5996
1000        5424
500         4059
600         2815
700         2030
800         1594
2000        1521
900         1320
3000         675
4000         348
5000         227
10000        139
6000         136
7000          91
8000          59
9000          56
20000         14
50000          2
30

In [25]:
# Extraire les catégories uniques
df_categories = uk[['categoryName']].drop_duplicates().reset_index(drop=True)

# Générer un identifiant unique pour chaque catégorie
df_categories['category_id'] = df_categories.index.astype(str).str.zfill(6)  # Par exemple, '000001'

# Réorganiser les colonnes
df_categories = df_categories[['category_id', 'categoryName']]

# Renommer les colonnes pour correspondre au schéma
df_categories.rename(columns={'categoryName': 'category_name'}, inplace=True)

# Afficher un aperçu
print(df_categories.head())


  category_id            category_name
0      000000           Hi-Fi Speakers
1      000001  CD, Disc & Tape Players
2      000002      Wearable Technology
3      000003              Light Bulbs
4      000004        Bathroom Lighting


##  Création de la Table Categories


In [26]:
# Création de la table Categories
df_categories = df_amz_uk[['categoryName']].drop_duplicates().reset_index(drop=True)
df_categories['category_id'] = df_categories.index.astype(str).str.zfill(6)
df_categories = df_categories[['category_id', 'categoryName']]
df_categories.rename(columns={'categoryName': 'category_name'}, inplace=True)

# Afficher un aperçu
print(df_categories.head())


  category_id            category_name
0      000000           Hi-Fi Speakers
1      000001  CD, Disc & Tape Players
2      000002      Wearable Technology
3      000003              Light Bulbs
4      000004        Bathroom Lighting


## Création de la Table Products

In [27]:
df_amz_uk.columns

Index(['asin', 'title', 'imgUrl', 'productURL', 'stars', 'reviews', 'price',
       'isBestSeller', 'boughtInLastMonth', 'categoryName'],
      dtype='object')

In [28]:
# Fusionner en spécifiant les colonnes respectives
df_products = df_amz_uk.merge(
    df_categories,
    left_on='categoryName',
    right_on='category_name',
    how='left'
)

# Vérifier s'il y a des correspondances manquantes
missing_categories = df_products[df_products['category_id'].isnull()]
print(f"Nombre de produits sans catégorie correspondante : {len(missing_categories)}")

# Si nécessaire, gérer les catégories manquantes
if len(missing_categories) > 0:
    print("Certaines catégories n'ont pas de correspondance. Vous pouvez les traiter comme suit :")
    # Exemple : attribuer une catégorie par défaut
    df_products['category_id'].fillna('000000', inplace=True)

# Sélectionner les colonnes pertinentes
df_products = df_products[['asin', 'title', 'imgUrl', 'productURL', 'price', 'isBestSeller', 'category_id']]

# Renommer les colonnes pour correspondre au schéma
df_products.rename(columns={
    'imgUrl': 'img_url',
    'productURL': 'product_url',
    'isBestSeller': 'is_best_seller'
}, inplace=True)

# Supprimer les doublons si nécessaire
df_products.drop_duplicates(subset=['asin'], inplace=True)

# Afficher un aperçu des données
print(df_products.head())



Nombre de produits sans catégorie correspondante : 0
         asin                                              title  \
0  B09B96TG33  Echo Dot (5th generation, 2022 release) | Big ...   
1  B01HTH3C8S  Anker Soundcore mini, Super-Portable Bluetooth...   
2  B09B8YWXDF  Echo Dot (5th generation, 2022 release) | Big ...   
3  B09B8T5VGV  Echo Dot with clock (5th generation, 2022 rele...   
4  B09WX6QD65  Introducing Echo Pop | Full sound compact Wi-F...   

                                             img_url  \
0  https://m.media-amazon.com/images/I/71C3lbbeLs...   
1  https://m.media-amazon.com/images/I/61c5rSxwP0...   
2  https://m.media-amazon.com/images/I/61j3SEUjMJ...   
3  https://m.media-amazon.com/images/I/71yf6yTNWS...   
4  https://m.media-amazon.com/images/I/613dEoF9-r...   

                              product_url  price  is_best_seller category_id  
0  https://www.amazon.co.uk/dp/B09B96TG33  21.99           False      000000  
1  https://www.amazon.co.uk/dp/B01HTH3C8S  

In [29]:
# Vérifier que tous les 'category_id' dans 'Products' existent dans 'Categories'
missing_categories = df_products[df_products['category_id'].isnull()]
print(f"Nombre de produits sans catégorie correspondante : {len(missing_categories)}")

if len(missing_categories) == 0:
    print("Toutes les catégories sont correctement associées.")
else:
    print("Certaines catégories sont manquantes. Vérifiez les données.")


Nombre de produits sans catégorie correspondante : 0
Toutes les catégories sont correctement associées.


## Création de la Table Rating

In [30]:
# Création de la table Ratings
df_ratings = df_amz_uk[['asin', 'stars', 'reviews']].copy()
df_ratings.drop_duplicates(subset=['asin'], inplace=True)

# Afficher un aperçu
print(df_ratings.head())


         asin  stars  reviews
0  B09B96TG33    4.7    15308
1  B01HTH3C8S    4.7    98099
2  B09B8YWXDF    4.7    15308
3  B09B8T5VGV    4.7     7205
4  B09WX6QD65    4.6     1881


## Création de la Table Sales

In [31]:
# Création de la table Sales
df_sales = df_amz_uk[['asin', 'boughtInLastMonth']].copy()
df_sales.rename(columns={'boughtInLastMonth': 'bought_in_last_month'}, inplace=True)
df_sales.drop_duplicates(subset=['asin'], inplace=True)

# Afficher un aperçu
print(df_sales.head())


         asin  bought_in_last_month
0  B09B96TG33                     0
1  B01HTH3C8S                     0
2  B09B8YWXDF                     0
3  B09B8T5VGV                     0
4  B09WX6QD65                     0


## Init BQ


In [38]:
# Initialiser le client BigQuery avec le fichier de clé JSON
client = bigquery.Client.from_service_account_json(service_account_path)

# Définir le projet et le dataset
project_id = 'bot-recommendation'  # Remplacez par votre ID de projet
dataset_id = 'amazonuk_data'
dataset_ref = client.dataset(dataset_id, project=project_id)

# Exemple de vérification : Lister les tables dans le dataset
tables = client.list_tables(dataset_ref)
print(f"Tables dans le dataset {dataset_id}:")
for table in tables:
    print(table.table_id)

Tables dans le dataset amazonuk_data:


## definition du schéma 

In [39]:
# Définition des schémas
schema_categories = [
    bigquery.SchemaField("category_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("category_name", "STRING", mode="REQUIRED"),
]

schema_products = [
    bigquery.SchemaField("asin", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("title", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("img_url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("product_url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("price", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("is_best_seller", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("category_id", "STRING", mode="REQUIRED"),
]

schema_ratings = [
    bigquery.SchemaField("asin", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("stars", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("reviews", "INTEGER", mode="NULLABLE"),
]

schema_sales = [
    bigquery.SchemaField("asin", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("bought_in_last_month", "INTEGER", mode="NULLABLE"),
]

# Fonction pour créer une table
def create_table(table_name, schema):
    table_ref = dataset_ref.table(table_name)
    try:
        table = bigquery.Table(table_ref, schema=schema)
        table = client.create_table(table)
        print(f"Table {table_name} créée.")
    except Exception as e:
        print(f"Erreur lors de la création de la table {table_name}: {e}")

# Créer les tables
create_table('Categories', schema_categories)
create_table('Products', schema_products)
create_table('Ratings', schema_ratings)
create_table('Sales', schema_sales)

# Fonction pour charger un DataFrame dans BigQuery
def load_dataframe_to_bq(df, table_name):
    table_ref = dataset_ref.table(table_name)
    job = client.load_table_from_dataframe(
        df, table_ref
    )
    job.result()  # Attendre la fin du job
    print(f"Données chargées dans la table {table_name}.")

# Charger les données
load_dataframe_to_bq(df_categories, 'Categories')
load_dataframe_to_bq(df_products, 'Products')
load_dataframe_to_bq(df_ratings, 'Ratings')
load_dataframe_to_bq(df_sales, 'Sales')

Table Categories créée.
Table Products créée.
Table Ratings créée.
Table Sales créée.
Données chargées dans la table Categories.
Données chargées dans la table Products.
Données chargées dans la table Ratings.
Données chargées dans la table Sales.
